In [24]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
from math import radians, sin, cos, sqrt, atan2

# utils
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from optuna.integration import LightGBMPruningCallback
from lightgbm import LGBMRegressor
import lightgbm as lgb
import optuna

import eli5
from eli5.sklearn import PermutationImportance
import json


##### Data Load

In [25]:
# 파일 불러오기
TRAIN_FE_PATH = "./Data/02_feature_end/train_fe.pkl"
TEST_FE_PATH  = "./Data/02_feature_end/test_fe.pkl"

train_df = pd.read_pickle(TRAIN_FE_PATH)
test_df  = pd.read_pickle(TEST_FE_PATH)

print("train_df shape:", train_df.shape)
print("test_df  shape:", test_df.shape)

print("\ntrain_df 컬럼 예시:", list(train_df.columns)[:25])

print("\n[train_df is_test 분포]")
print(train_df["is_test"].value_counts(dropna=False))

print("\n[Target 값 기초 통계]")
print(train_df["target"].describe())

train_df shape: (1118822, 127)
test_df  shape: (9272, 127)

train_df 컬럼 예시: ['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사']

[train_df is_test 분포]
is_test
0    1118822
Name: count, dtype: int64

[Target 값 기초 통계]
count    1.118822e+06
mean     5.799153e+04
std      4.642602e+04
min      3.500000e+02
25%      3.050000e+04
50%      4.480000e+04
75%      6.980000e+04
max      1.450000e+06
Name: target, dtype: float64


##### Feature / Target 분리하기

In [26]:
# Feature / Target 분리하기 원본
TARGET_COL = "target"

# 모델에 넣지 않을 컬럼들
NON_FEATURE_COLS = [
    TARGET_COL,
    "is_test",
    # 만약 id, row_id 같은 컬럼 있으면 여기 추가
]

# 피처 컬럼 리스트
orig_feature_cols = [c for c in train_df.columns if c not in NON_FEATURE_COLS]

'''
# 삭제 후보
interest_features = ["base_rate", "rate_mom", "rate_yoy", "rate_bucket"]

for f in interest_features:
    if f in train_df.columns and f not in orig_feature_cols:
        orig_feature_cols.append(f)
'''

print("orig_feature_cols 개수:", len(orig_feature_cols))

print("피처 개수:", len(orig_feature_cols))
print("피처 예시 20개:", orig_feature_cols[:20])
print("피처 110개:", orig_feature_cols[:110])

# X, y 생성
X = train_df[orig_feature_cols].copy()
y = train_df[TARGET_COL].copy()

print("\nX shape:", X.shape)
print("y shape:", y.shape)

orig_feature_cols 개수: 125
피처 개수: 125
피처 예시 20개: ['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형']
피처 110개: ['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '좌표X', '좌표Y', '단지신청일', '본번_missing', '부번_missing', '아파트명_missing', '번지_missing', 'has_cancelled', '시', '구', '동', '구_평균가격', '구_중위가격', '구_거래량', '동_평균가격', '동_거래량', '강남여부', 'CBD_여부', 'GBD_여부', 'YBD_여부', 'subway_min_dist_km', 'subway_is_500m', 'subway_cnt_500m', 'subway_cnt_1km', 'bus_mi

##### LGBM 용 Safe 컬럼명 생성 + 문자열 Factorize
- 한글/특수문자 컬럼명을 모델이 학습하기 좋은 환경인 알파벳/숫자 형식으로 변경

In [27]:
def make_base_safe_name(name: str) -> str:
    # 1) 한글/특수문자 → _ 로 치환
    safe = re.sub(r"[^0-9a-zA-Z_]", "_", name)
    # 2) 첫 글자가 숫자면 접두어 붙이기
    if re.match(r"^[0-9]", safe):
        safe = "f_" + safe
    return safe

safe_name_counts = {}
safe_feature_cols = []

for col in orig_feature_cols:
    base = make_base_safe_name(col)
    if base in safe_name_counts:
        safe_name_counts[base] += 1
        safe = f"{base}__{safe_name_counts[base]}"
    else:
        safe_name_counts[base] = 0
        safe = base
    safe_feature_cols.append(safe)

# 매핑 딕셔너리
orig2safe = dict(zip(orig_feature_cols, safe_feature_cols))
safe2orig = dict(zip(safe_feature_cols, orig_feature_cols))

# X를 safe 이름으로 변경
X_safe = X.rename(columns=orig2safe)

print("safe 컬럼 개수:", len(safe_feature_cols))
print("safe 유니크 개수:", len(set(safe_feature_cols)))

dups = [n for n in safe_feature_cols if safe_feature_cols.count(n) > 1]
print("중복 safe 이름 개수:", len(set(dups)))

print("\n[원래 이름 → safe 이름 매핑 예시 10개]")
for o, s in list(orig2safe.items())[:10]:
    print(f"{o}  -->  {s}")

safe 컬럼 개수: 125
safe 유니크 개수: 125
중복 safe 이름 개수: 0

[원래 이름 → safe 이름 매핑 예시 10개]
시군구  -->  ___
번지  -->  __
본번  -->  ____1
부번  -->  ____2
아파트명  -->  ____
전용면적  -->  ______1
계약년월  -->  ______2
계약일  -->  _____1
층  -->  _
건축년도  -->  ______3


In [28]:
# 문자열(범주형) 컬럼 safe 이름 기준으로 찾기
cat_cols_safe = [
    orig2safe[c]
    for c in orig_feature_cols
    if X[c].dtype == "object"
]

print("범주형(문자열) 컬럼 수:", len(cat_cols_safe))
print("범주형(문자열) 예시:", cat_cols_safe[:10])

# factorize 인코딩
for c in cat_cols_safe:
    X_safe[c], _ = pd.factorize(X_safe[c], sort=True)

print("\n인코딩 후 dtype 분포:")
print(X_safe.dtypes.value_counts())

범주형(문자열) 컬럼 수: 34
범주형(문자열) 예시: ['___', '__', '____1', '____2', '____', '_____2', '______', '______4', '________1', 'k_________________']

인코딩 후 dtype 분포:
int64       66
float64     57
category     1
category     1
Name: count, dtype: int64


##### Train / Valid Split

In [29]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_safe,
    y,
    test_size=0.2,
    random_state=42
)

print("X_train shape:", X_train.shape)
print("X_valid shape:", X_valid.shape)
print("y_train shape:", y_train.shape)
print("y_valid shape:", y_valid.shape)

X_train shape: (895057, 125)
X_valid shape: (223765, 125)
y_train shape: (895057,)
y_valid shape: (223765,)


##### LightGBM 학습 및 RMSE 측정 및 출력

In [ ]:
# 1. Optuna objective 함수 정의 (튜닝용)
#    - num_boost_round: 1000 (튜닝용으로만 줄임)
#    - PruningCallback 추가 → 성능 안 좋은 trial은 중간에 컷

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "random_state": 42,
        "num_threads": -1,

        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 255),
        "max_depth": trial.suggest_int("max_depth", -1, 16),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 10),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0),
    }

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)

    # 🔹 여기서 valid 세트 이름을 "valid"로 명시!
    pruning_callback = LightGBMPruningCallback(trial, "rmse", "valid")

    model = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[valid_data],
        valid_names=["valid"],   # ← 여기에 맞춰 "valid"라고 지정했으니까
        callbacks=[
            lgb.early_stopping(stopping_rounds=100),
            lgb.log_evaluation(100),
            pruning_callback,
        ],
    )

    y_valid_pred = model.predict(X_valid, num_iteration=model.best_iteration)

    mse = mean_squared_error(y_valid, y_valid_pred)
    rmse = np.sqrt(mse)
    return rmse

# 2. Optuna Study 실행
print("\n[Optuna 하이퍼파라미터 튜닝 시작]")
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=42),
)
study.optimize(objective, n_trials=30)   # 💡 우선 20~30 trial로 가볍게

print("\n[Optuna 결과]")
print("Best RMSE:", study.best_value)
print("Best params:", study.best_params)

'\n# 1. Optuna objective 함수 정의 (튜닝용)\n#    - num_boost_round: 1000 (튜닝용으로만 줄임)\n#    - PruningCallback 추가 → 성능 안 좋은 trial은 중간에 컷\n\ndef objective(trial):\n    params = {\n        "objective": "regression",\n        "metric": "rmse",\n        "boosting_type": "gbdt",\n        "verbosity": -1,\n        "random_state": 42,\n        "num_threads": -1,\n\n        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),\n        "num_leaves": trial.suggest_int("num_leaves", 31, 255),\n        "max_depth": trial.suggest_int("max_depth", -1, 16),\n        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),\n        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),\n        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),\n        "bagging_freq": trial.suggest_int("bagging_freq", 0, 10),\n        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),\n        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0),\

In [31]:
best_params = {
    'learning_rate': 0.030710573677773714,
    'num_leaves': 244,
    'max_depth': 12,
    'min_data_in_leaf': 188,
    'feature_fraction': 0.6624074561769746,
    'bagging_fraction': 0.662397808134481,
    'bagging_freq': 0,
    'lambda_l1': 4.330880728874676,
    'lambda_l2': 3.005575058716044
}


In [32]:
final_params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "verbosity": -1,
    "random_state": 42,
    "num_threads": -1,
}
final_params.update(best_params)

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

print("\n[최종 모델 학습 시작]")
final_model = lgb.train(
    final_params,
    train_data,
    num_boost_round=2000,               
    valid_sets=[valid_data],
    valid_names=["valid"],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(50),
    ],
)

y_pred = final_model.predict(X_valid, num_iteration=final_model.best_iteration)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f"[최종 모델 RMSE]: {rmse:.4f}")
print("Best iteration:", final_model.best_iteration)



[최종 모델 학습 시작]
Training until validation scores don't improve for 100 rounds
[50]	valid's rmse: 16447.4
[100]	valid's rmse: 10647.2
[150]	valid's rmse: 9068.67
[200]	valid's rmse: 8438.89
[250]	valid's rmse: 8098.37
[300]	valid's rmse: 7833.43
[350]	valid's rmse: 7630.18
[400]	valid's rmse: 7474.93
[450]	valid's rmse: 7326.72
[500]	valid's rmse: 7201.86
[550]	valid's rmse: 7088.27
[600]	valid's rmse: 6997.85
[650]	valid's rmse: 6923.66
[700]	valid's rmse: 6840.09
[750]	valid's rmse: 6771.72
[800]	valid's rmse: 6705
[850]	valid's rmse: 6650.32
[900]	valid's rmse: 6604.61
[950]	valid's rmse: 6555.24
[1000]	valid's rmse: 6496.34
[1050]	valid's rmse: 6458.81
[1100]	valid's rmse: 6418.18
[1150]	valid's rmse: 6383.3
[1200]	valid's rmse: 6347.81
[1250]	valid's rmse: 6311.6
[1300]	valid's rmse: 6281.26
[1350]	valid's rmse: 6247.77
[1400]	valid's rmse: 6220.75
[1450]	valid's rmse: 6194.04
[1500]	valid's rmse: 6174.76
[1550]	valid's rmse: 6156.67
[1600]	valid's rmse: 6141.23
[1650]	valid's rmse:

In [33]:
# 우리가 만든 대표 입지 파생변수들을 중심으로 한글 이름 매핑
rename_dict = {
    # 입지 / 가격 통계
    "동_평균가격": "동별 평균 실거래가",
    "동_거래량": "동별 거래량",
    "구_평균가격": "구별 평균 실거래가",
    "구_중위가격": "구별 중위 실거래가",
    "구_거래량": "구별 거래량",

    # 강남 / 업무지구
    "강남여부": "강남 4구 여부",
    "CBD_여부": "도심(CBD) 여부",
    "GBD_여부": "강남권(GBD) 여부",
    "YBD_여부": "여의도(YBD) 여부",

    # 지하철
    "subway_min_dist_km": "가장 가까운 지하철역 거리(km)",
    "subway_cnt_500m": "500m 내 지하철역 수",
    "subway_cnt_1km": "1km 내 지하철역 수",
    "subway_is_500m": "지하철 500m 이내 여부",

    # 버스
    "bus_min_dist_km": "가장 가까운 버스정류장 거리(km)",
    "bus_cnt_300m": "300m 내 버스정류장 수",
    "bus_cnt_500m": "500m 내 버스정류장 수",
    "bus_is_300m": "버스 300m 이내 여부",
    "bus_is_500m": "버스 500m 이내 여부",

    # 학교 거리
    "elem_min_dist_km": "가장 가까운 초등학교 거리(km)",
    "mid_min_dist_km": "가장 가까운 중학교 거리(km)",
    "high_min_dist_km": "가장 가까운 고등학교 거리(km)",
    "school_min_dist_km": "가장 가까운 학교 거리(km)",

    # 구별 학교 수
    "elem_school_cnt": "구별 초등학교 수",
    "mid_school_cnt": "구별 중학교 수",
    "high_school_cnt": "구별 고등학교 수",

    # 건축년도 및 연식 관련
    "building_age": "건물나이",
    "building_age_bucket": "노후도구간",
    "year_group": "건축년도_구간",
    "year_sq": "건축년도_제곱",
    "year_sqrt": "건축년도_루트",

    # 층수 관련
    "floor_bucket": "층수_구간",
    "floor_abs": "층_절댓값",
    "is_low_floor": "저층",
    "is_mid_floor": "중층",
    "is_high_floor": "고층",

    # 단지 기반
    "max_floor_in_complex": "단지_최고층",
    "is_top_floor": "최상층",
    "complex_scale": "단지규모",

    # 지역 기반
    "avg_age_by_dong": "동_평균연식",
    "avg_floor_by_dong": "동_평균층수",
    "ratio_new_buildings": "동_신축비율",

    # 평 기반
    "log_area": "전용면적_log",
    "area_pyeong": "평수",
    "area_bucket": "평수구간",
    "price_per_pyeong": "평당가격",
    "dong_avg_pyp": "동별 평균 평당가격",

    # 거래 시점
    "contract_year": "계약연도",
    "contract_month": "계약월",
    "contract_quarter": "계약분기",
    "season": "계약계절",
    "yearly_price_growth": "구_연간가격증가율",

    # 거래 특성
    "same_apt_area_total_trades": "동일단지면적_총거래수",
    "same_apt_area_past_trades": "동일단지면적_과거거래수",

    # 아파트 이름 기반 텍스트
    "is_brand_apt": "브랜드아파트여부",
    "has_redev_keyword": "재건축키워드여부",
    "has_scale_keyword": "규모키워드여부",

    # 금리
    "base_rate": "기준금리",
    "base_mom": "기준금리_전월변화",
    "base_yoy": "기준금리_전년동월변화",

    "mortgage_rate": "주담대금리",
    "mortgage_mom": "주담대금리_전월변화",
    "mortgage_yoy": "주담대금리_전년동월변화",

    "spread": "대출-기준금리_스프레드",
    "spread_mom": "스프레드_전월변화",

    "mortgage_ma3": "주담대금리_3개월이동평균",
    "mortgage_ma6": "주담대금리_6개월이동평균",

    "mortgage_lag1": "주담대금리_L1",
    "mortgage_lag3": "주담대금리_L3",
    "mortgage_lag6": "주담대금리_L6",

    "base_rate_bucket": "기준금리_구간",
    "mortgage_rate_bucket": "주담대금리_구간",

    # 기타 (기존 변수들도 필요하면 여기 추가)
    "전용면적": "전용면적(㎡)",
    "계약년월": "계약연월",
    "계약일": "계약일",
    "건축년도": "건축년도",
    "층": "해당 층수",


}

In [34]:
# 1. final_model에서 feature importance 추출 (safe 이름 기준)
importance_gain = final_model.feature_importance(importance_type="gain")
importance_split = final_model.feature_importance(importance_type="split")
feature_safe = final_model.feature_name()   # ← safe 이름들

# 2. DataFrame 생성 (safe 이름 기준)
fi_df = pd.DataFrame({
    "feature_safe": feature_safe,
    "importance_gain": importance_gain,
    "importance_split": importance_split,
})

# 3. safe → 원래 컬럼명으로 매핑 (위에서 만든 safe2orig 사용)
fi_df["feature"] = fi_df["feature_safe"].map(safe2orig)

# 4. 중요도 순으로 정렬
fi_df = fi_df.sort_values("importance_gain", ascending=False).reset_index(drop=True)

# 5. 사람이 읽기 좋은 한글 라벨 (rename_dict 사용)
fi_df["feature_readable"] = fi_df["feature"].map(rename_dict)
fi_df["feature_readable"] = fi_df["feature_readable"].fillna(fi_df["feature"])

print("\n[Feature Importance 상위 20개]")
print(fi_df[["feature", "feature_readable", "importance_gain"]].head(20))



[Feature Importance 상위 20개]
                       feature feature_readable  importance_gain
0                       동_평균가격       동별 평균 실거래가     7.456475e+15
1                         전용면적          전용면적(㎡)     4.693366e+15
2                         계약년월             계약연월     4.042798e+15
3                 dong_avg_pyp       동별 평균 평당가격     3.803675e+15
4                     log_area         전용면적_log     2.357958e+15
5                contract_year             계약연도     1.200348e+15
6                       구_평균가격       구별 평균 실거래가     9.840436e+14
7                  area_pyeong               평수     9.637072e+14
8                         건축년도             건축년도     7.647605e+14
9         max_floor_in_complex           단지_최고층     4.883796e+14
10                mortgage_ma6    주담대금리_6개월이동평균     2.704190e+14
11  same_apt_area_total_trades      동일단지면적_총거래수     2.580426e+14
12                building_age             건물나이     2.545086e+14
13                   base_rate             기준금리     2.270820e

In [ ]:
OUTPUT_DIR = "./Data/03_model_end"
os.makedirs(OUTPUT_DIR, exist_ok=True)

FI_PATH = os.path.join(OUTPUT_DIR, "feature_importance_gain.csv")
fi_df.to_csv(FI_PATH, index=False, encoding="utf-8-sig")
print("Feature Importance 저장 완료:", FI_PATH)

✅ Feature Importance 저장 완료: ./Data/03_model_end/feature_importance_gain.csv


In [ ]:
TOP_N = 60
top_features = fi_df["feature"].head(TOP_N).tolist()

TOP_FEATURES_PATH = os.path.join(OUTPUT_DIR, f"top{TOP_N}_features.json")
with open(TOP_FEATURES_PATH, "w", encoding="utf-8") as f:
    json.dump(top_features, f, ensure_ascii=False)

print(f"Top{TOP_N} feature 리스트 저장 완료:", TOP_FEATURES_PATH)

✅ Top60 feature 리스트 저장 완료: ./Data/03_model_end/top60_features.json
